Variable Coefficients
===
$\DeclareMathOperator{\opdiv}{div}$

A setup with varying heat-conductivity is modeled by the equation

$$
-\opdiv \lambda(x) \nabla u(x) = f(x),
$$

where $\lambda$ is the heat-conductivity, relating the heat-flux 

$$
q = -\lambda \nabla u
$$

to the temperature gradient $\nabla u$.

In the case of discontinuous $\lambda$, the equation is understood in distributional sense. This includes the interface conditions: The temperature on the left and right side are equal, and the heat flux leaving the left side enters the right side:

\begin{eqnarray*}
u_l & = & u_r \\
\lambda_l \frac{\partial u_r}{\partial n} & = & \lambda_r \frac{\partial u_r}{\partial n}
\end{eqnarray*}

The variational form is: find $u \in H^1(\Omega)$

$$
\int_\Omega \lambda(x) \nabla u \nabla v = \int_\Omega f v
$$

There is no issue with discontinuous coefficients. Both interface conditions are included: 
* continuity of temperature $u$ by the continuity of the trial space
* continuity of the heat flux in weak sense, such as Neumann boundary conditions

In [ ]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.webgui import Draw as DrawGeo

Make a 2D geometry (with brand-new Opencascade Technology):

In [ ]:
from netgen.occ import *
box = Rectangle(1,1).Face()
circ = Circle((0.3,0.7), 0.1).Face()
bar = MoveTo(0.2,0.2).Rectangle(0.7,0.1).Face()
air = box-circ-bar

circ.faces.name = "source"
air.faces.name = "air"
bar.faces.name = "bar"
air.edges.Min(Y).name ='b'
air.edges.Max(X).name ='r'

shape = Glue([air,circ,bar])
DrawGeo (shape);

In [ ]:
geo = OCCGeometry(shape, dim=2)
mesh = Mesh(geo.GenerateMesh(maxh=0.05)).Curve(3)
Draw (mesh);

In [ ]:
print (mesh.GetMaterials())

In [ ]:
print (mesh.GetBoundaries())

In [ ]:
fes = H1(mesh, order=3, dirichlet="b|r")
u = fes.TrialFunction()
v = fes.TestFunction()

make a coefficient function taking one constant per material. Uses Python list comprehension.

In [ ]:
lam = mesh.MaterialCF({ "air" : 1, "bar" : 100, "source" : 2 } )
Draw (lam, mesh, "lambda");

use conductivity in bilinear-form:

In [ ]:
a = BilinearForm(fes)
a += lam*grad(u)*grad(v)*dx

f = LinearForm(fes)
f += 50*v*dx("source")
a.Assemble()
f.Assemble();

In [ ]:
reg = mesh.Materials("source|bar")
print (reg.VB(), reg.Mask())

In [ ]:
gfu = GridFunction(fes)
gfu.vec.data = a.mat.Inverse(fes.FreeDofs()) * f.vec

In [ ]:
Draw (gfu, mesh, "temperature");

In [ ]:
Draw (grad(gfu), mesh, "gradient");

In [ ]:
Draw (-lam*grad(gfu), mesh, "heatflux");